# I. Importando banco de dados

In [77]:
#Importando bibliotecas
import scipy
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

plt.style.use("ggplot")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
#Importando dataset
df = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
df.head()

Verificando valores faltantes 

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T

In [ ]:
df.info()

## Removendo outliers

### Função

In [78]:
#Remove outliers
def remove_outliers(dataframe, colunas):
    
    for i in colunas:
        mediana = np.median(dataframe[i])
        Q3 = dataframe[i].quantile(0.75)
        Q1 = dataframe[i].quantile(0.25)

        IQR = Q3 - Q1

        limite_inferior = Q1 - (IQR * 1.5)
        limite_superior = Q3 + (IQR * 1.5)
        
        dataframe.loc[(dataframe[i] < limite_inferior) | (dataframe[i] > limite_superior), i] = mediana

Boxplot pós remoção

In [79]:
for i in range(5):
    remove_outliers(df, columns)
    
fig, a = plt.subplots(3, figsize=(15,15))

sns.boxplot(x= df["fixed acidity"], ax = a[0], color="Blue")
sns.boxplot(x= df["volatile acidity"], ax = a[1], color="Blue")
sns.boxplot(x= df["residual sugar"], ax = a[2], color="Blue")

In [81]:
fig, ax = plt.subplots(4, figsize=(15,20))

for i in range(4):             
        sns.boxplot(x= df[columns[j]], ax = ax[i])
        j += 1

Boxplot pós remoção

In [82]:
fig, ax2 = plt.subplots(3, figsize=(15,20))

for i in range(3):             
        sns.boxplot(x= df[columns[j]], ax = ax2[i])
        j += 1

Boxplot pós remoção

# II. Visualizando e entendendo o dataset

In [83]:
plt.figure(figsize=(13,8))
sns.heatmap(df.corr(), annot= True, cmap= "Reds", vmin= -1, vmax= 1)
plt.title("Correlation map")

Relações interessantes do heat map:

* O "alcool" é um fator que pode determinar, um pouco, a qualidade do seu vinho.
* O "fixed acidity" e o "citric acid" são substancias muito correlacionadas, elas afetam outras duas caracteristicas do vinho, a " e o "pH". A relação entre a acidez do vinho e a densidade é proporcional, ou seja, quanto mais acido for o vinho maior será a sua densidade, já a relação entre a acidez e o pH é inversamente proporcional, ou seja, quanto mais acido for o vinho menor será o seu pH, isso ocorre pelo fato de que substâncias ácidas têm um pH menor.
* Vinhos com um alto teor alcoólico são menos densos.

In [84]:
sns.scatterplot(y= df["citric acid"], x= df["fixed acidity"], hue= df.quality)

In [85]:
sns.scatterplot(y= df["density"], x= df["fixed acidity"], hue= df.quality)

In [86]:
sns.scatterplot(y= df["pH"], x= df["fixed acidity"], hue= df.quality)

In [87]:
sns.scatterplot(y= df["density"], x= df["alcohol"], hue= df.quality)

In [89]:
sns.countplot(x = df.quality)

## Relação entre a qualidade do vinho e suas caracteristicas

In [90]:
fig, ax1 = plt.subplots(4, figsize=(15,20))
columns = list(df.drop(columns= ['quality']).columns)
k = 0

for i in range(4):          
        sns.boxplot(x = df.quality, y= df[columns[k]], ax = ax1[i])
        k += 1
        

O fator que impacta significativamente a qualidade do vinho é o "volatile acidity", quanto menor for o melhor será o vinho.


In [91]:
fig, ax = plt.subplots(4, figsize=(15,20))

for i in range(4):             
        sns.boxplot(x = df.quality, y= df[columns[k]], ax = ax[i])
        k += 1

In [92]:
fig, ax2 = plt.subplots(3, figsize=(15,20))

for i in range(3):             
        sns.boxplot(x = df.quality, y= df[columns[k]], ax = ax2[i])
        k += 1

Como foi visto no heatmap, vinhos com um alto teor alcoólico costumam ter uma alta qualidade, porém isso não significa que o alcool seja um fator desicivo para a qualidade do vinho, visto que a qualidade depende de varios fatores, com isso, é possivel observar vinhos com um alto teor alcoólico e uma qualidade baixa.

In [ ]:
fig, ax1 = plt.subplots(3, 4, figsize=(20,17))
columns = list(df.columns)
k = 0

for i in range(3):
    for j in range(4):
        sns.distplot( df[columns[k]], ax = ax1[i][j])
        k += 1

# III. Modelos

In [93]:
# Preparando dados de qualidade
df['quality'] = ['good' if x>=6.5 else "bad" for x in df['quality']]

previsores = df.drop(columns="quality").values
classe = df["quality"].values

X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores, classe, test_size = 0.3, random_state = 0)
valor = []

## Random Forest

In [94]:
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treinamento, y_treinamento)
previsoes = floresta.predict(X_teste)

score = accuracy_score(y_teste, previsoes)

print(f"Matriz confusão: \n{confusion_matrix(y_teste, previsoes)}")
print(f"\n\nAccuracy: {accuracy_score(y_teste, previsoes)}")

valor.append(("Random Forest Classifier", score))

## Naive Bayes

In [95]:
nb = GaussianNB()
nb.fit(X_treinamento, y_treinamento)
previsoes = nb.predict(X_teste)

score = accuracy_score(y_teste, previsoes)

print(f"Matriz confusão: \n{confusion_matrix(y_teste, previsoes)}")
print(f"\n\nAccuracy: {accuracy_score(y_teste, previsoes)}")

valor.append(("Naive Bayes",score))

## KNeighbors

In [96]:
neighbors = KNeighborsClassifier(n_neighbors=3)
neighbors.fit(X_treinamento, y_treinamento)
previsoes = neighbors.predict(X_teste)

score = accuracy_score(y_teste, previsoes)

print(f"Matriz confusão: \n{confusion_matrix(y_teste, previsoes)}")
print(f"\n\nAccuracy: {accuracy_score(y_teste, previsoes)}")

valor.append(("KNeighbors Classifier", score))

## Decision Tree

In [97]:
tree = DecisionTreeClassifier(random_state=1)
tree.fit(X_treinamento, y_treinamento)
previsoes = tree.predict(X_teste)

score = accuracy_score(y_teste, previsoes)

print(f"Matriz confusão: \n{confusion_matrix(y_teste, previsoes)}")
print(f"\n\nAccuracy: {accuracy_score(y_teste, previsoes)}")

valor.append(("Decision Tree Classifier", score))

In [98]:
modelos = pd.DataFrame(valor, columns = ['Name', 'Accuracy'])
modelos.sort_values(by= 'Accuracy')

In [99]:
plt.figure(figsize=(8,5))
sns.barplot(y= modelos.Name, x= modelos.Accuracy, order= ["Random Forest Classifier", "Decision Tree Classifier", "KNeighbors Classifier", "Naive Bayes"])
plt.title("Modelos")